In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

<h1><center>Read PE data from GE data</center></h1>

In [2]:
data_folder = os.getcwd()+'\\Data'
data = {'Production_v': data_folder+'\\Production_v.xlsx', 'Production_p': data_folder+'\\Production_p.xlsx'}
db = ReadData.read_data.main(data)

This example outlines how the function 'PE_from_GE' from ReadData.py works. The current ipynb-file essentially executes the following:

In [3]:
s0 = 'a'
db_new = ReadData.PE_from_GE(db,s0)

The sector set $(s)$ is dropped, subsets defined over $s$ are dropped, variables and parameters are sliced according to $s$ as well.

**1: Sets**

*Alias':*

In [4]:
if 'alias_set' in db:
    if not (len(db['alias_set'])==1 and (db['alias_set']=='s').any()):
        db_new['alias_set'] = db['alias_set'][db['alias_set']!='s']
        db_new['alias_map2'] = db['alias_map2'][~db['alias_map2'].isin(db.alias_all['s'])]
        db_new['alias_'] = db['alias_'][db['alias_'].get_level_values(0)!='s']

*Sets:*

In [5]:
for set_ in (set(db.sets['sets'])-set(db.alias_all['s'])-set('s')-set(['alias_set','alias_map2'])):
    db_new[set_] = db[set_]

*Subsets and mappings:*

In [6]:
for set_ in db.sets['subsets']:
    if set_ not in db_new and db[set_].name!='s':
        db_new[set_] = db[set_]
for set_ in db.sets['mappings']:
    if set_!='alias_':
        db_new[set_] = db[set_] if 's' not in db[set_].names else db[set_][db[set_].get_level_values('s')!=s0].droplevel(level='s').unique()

**2: Variables/parameters**

Update variables:

In [7]:
for scalar in db.variables['scalar_variables']:
    db_new[scalar] = db[scalar]
for scalar in db.parameters['scalar_parameters']:
    db_new[scalar] = db[scalar]
for var in db.variables['variables']:
    db_new[var] = db[var] if 's' not in db[var].index.names else db[var][db[var].index.get_level_values('s')==s0].droplevel('s')
    db_new[var].attrs['type'] = 'variable'
for par in db.parameters['parameters']:
    db_new[par] = db[par] if 's' not in db[par].index.names else db[par][db[par].index.get_level_values('s')==s0].droplevel('s')
    db_new[par].attrs['type'] = 'parameter'    